<a href="https://colab.research.google.com/github/hululuzhu/chinese-ai-writing-share/blob/main/training/t5_finetune/2023_T5_Finetune_Chinese_Poem_Writing_V1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T5 写诗
- 设计：Pretrained T5 + “写诗 prompt” fine-tuning
  - 对比我的 [transformer training from scratch](https://github.com/hululuzhu/chinese-ai-writing-share/blob/main/%E4%B8%AD%E6%96%87%E5%86%99%E8%AF%97Transformer_Source_Code_Share_V1.ipynb)
  - 想要加入作者作为可选输入
    - 每个文章分两次输入，一次作者名字，一次“None”名字（通用）
- 数据：[诗歌github](https://github.com/chinese-poetry/chinese-poetry)
- 相关内容
  - [Huggingface](https://huggingface.co/)
  - LangZhou Chinese [MengZi T5 pretrained Model](https://huggingface.co/Langboat/mengzi-t5-base) and [paper](https://arxiv.org/pdf/2110.06696.pdf)
  - [SimpleT5 by Shivanandroy](https://github.com/Shivanandroy/simpleT5) (on top of pytorch and pytorch lightning) and [his awesome medium article](https://medium.com/geekculture/simplet5-train-t5-models-in-just-3-lines-of-code-by-shivanand-roy-2021-354df5ae46ba)
- 进度
  - 02/2023, improve source text to make it shorter and more concise
    - Enforce the alignment of text size in poems
  - Return max output len to 32 (instead 64 chars)
  - New model to be trained by 03/2023

## Load Data

In [35]:
!nvidia-smi

Wed Mar  1 02:50:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [36]:
IS_TEST_FLOW = False  #@param {type: "boolean"}

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import json
import urllib.request
import pandas as pd
!pip install -q "tqdm>=4.36.1" > /tmp/na
from tqdm.notebook import tqdm
!pip install -q chinese-converter > /tmp/na
import chinese_converter  # 繁体到简体需要
import pickle
import os
import pandas as pd
import numpy as np

In [39]:
# https://github.com/chinese-poetry/chinese-poetry
POEM_CONTENT = {
    'tang': {
        'total': 58,
        'pattern': "https://raw.githubusercontent.com/chinese-poetry/chinese-poetry/master/json/poet.tang.{0}.json"
    },
    'song': {
        'total': 255,
        'pattern': "https://raw.githubusercontent.com/chinese-poetry/chinese-poetry/master/json/poet.song.{0}.json"
    }
}

def get_poems(is_test=True, verbose=True):
  df_list = []
  for dynasty in POEM_CONTENT:
    size = 3 if is_test else POEM_CONTENT[dynasty]['total']
    pbar = tqdm(total=size, desc="Dynasty " + dynasty)
    for i in range(size):
      url = POEM_CONTENT[dynasty]['pattern'].format(i * 1000)
      if verbose:
        print(f"download {url} now")
      df_list.append(pd.read_json(url))
      pbar.update(1)
  return pd.concat(df_list)

In [40]:
df = get_poems(is_test=IS_TEST_FLOW, verbose=False)
df['concat_paragraphs'] = [''.join(map(str, l)) for l in df['paragraphs']]
df = df[['author', 'title', 'concat_paragraphs']]

def convert_schinese(tchinese):
  return chinese_converter.to_simplified(tchinese)

df['s_content'] = df.apply(lambda row: convert_schinese(''.join(row.concat_paragraphs)), axis=1)
df['s_title'] = df.apply(lambda row: convert_schinese(''.join(row.title)), axis=1)
df['s_author'] = df.apply(lambda row: convert_schinese(''.join(row.author)), axis=1)

my_df = df
print("my_df size", len(my_df))

Dynasty tang:   0%|          | 0/58 [00:00<?, ?it/s]

Dynasty song:   0%|          | 0/255 [00:00<?, ?it/s]

my_df size 311855


In [41]:
MAX_AUTHOR_CHAR = 4
MAX_TITLE_CHAR = 12
MIN_CONTENT_CHAR = 20
MAX_CONTENT_CHAR = 32
BAD_TOKENS = " ()[]《》（）□{}abcdefgxyz一"

def trim_author_fn(row):
  return row.s_author[:MAX_AUTHOR_CHAR]

def trim_title_fn(row):
  trimed_title = row.s_title[:MAX_TITLE_CHAR]
  for b in BAD_TOKENS:
    trimed_title = trimed_title.replace(b, "")
  return trimed_title

def trim_content_fn(row):
  trimed_content = row.s_content[:MAX_CONTENT_CHAR]
  # # End with a period to avoid partial ending to confuse model
  for b in BAD_TOKENS:
    trimed_content = trimed_content.replace(b, "")
  last_period = trimed_content.rfind("。")
  return trimed_content[:last_period+1]
  # return trimed_content

# Trim the size, a soft copy to avoid the view/copy conflict warning
my_df['s_author_trim'] = my_df.copy().apply(trim_author_fn, axis=1)
my_df['s_title_trim'] = my_df.copy().apply(trim_title_fn, axis=1)
my_df['s_content_trim'] = my_df.copy().apply(trim_content_fn, axis=1)

print("my_df size", len(my_df))

my_df size 311855


In [42]:
# Title cannot be empty
empty_title_mask = (my_df['s_title_trim'].str.len() == 0)
too_short_cotent_mask = (my_df['s_content_trim'].str.len() <= MIN_CONTENT_CHAR)
invalid_mask = (('无正文' == my_df['s_content_trim']) | ('无正文' == my_df['s_author_trim']))
too_short_mask =  empty_title_mask | too_short_cotent_mask | invalid_mask
# filtered_my_df = my_df.loc[too_short_mask]
# filtered_my_df

my_df = my_df.loc[~too_short_mask][[
  's_author_trim', 's_title_trim', 's_content_trim']]
print("my_df size", len(my_df))

my_df size 297836


In [43]:
import re
result_dict = {
    's_author_trim': [],
    's_title_trim': [],
    's_content_trim': [],
}
for i, row in my_df.iterrows():
  c = row['s_content_trim']
  snippets = list(re.split('，|。|？', c))
  lens = [len(s) for s in snippets if s.strip() != '']
  if max(lens) != min(lens) or max(lens) not in [5, 7]:
    continue
  result_dict['s_author_trim'].append(row['s_author_trim'])
  result_dict['s_title_trim'].append(row['s_title_trim'])
  result_dict['s_content_trim'].append(c)
# print("get rid of ", sum(bad_items))
my_df = pd.DataFrame(data=result_dict)
print("left", len(my_df))

left 225860


In [44]:
my_df.sample(100)

,s_author_trim,s_title_trim,s_content_trim
194058,李龏,关山月,玉门关外月，远接黑山明。晃色同沙冷，飞光掩雪清。
136881,刘应时,山居三首其二,小小园林畔，萧萧丛薄间。茅檐颇幽隠，竹径自回环。
22349,顾非熊,关试后嘉会里闻蝉感怀呈主,昔闻惊节换，常抱异乡愁。今听当命遂，方欢上国游。
96994,释文准,偈十二首其二,八月九月天，白露寒露节。门外在处山，秋风落黄叶。
48047,释重显,风幡竞辨其,不是幡兮不是风，衲僧于此作流通。渡河用筏寻常事，南山烧炭北山红。
...,...,...,...
152717,朱熹,崇真观,磴道千寻风满林，洞门无锁下秋荫。紫台凤去天关远，丹井龙归地轴深。
213504,方一夔,秋晚杂兴十二首其六,地占繁雄郡，人奔财赋疆。北僧泥佛相，南客贾胡装。
221879,罗公升,陵州,五月征裘路五千，长安疑在北辰边。故园归夢西风冷，明日陵州正觅船。
201411,释可湘,船子赞,药山之子石头孙，饥过心荒浪里奔。得箇锦鳞如许大，看来当与道吾分。


In [45]:
AUTHOR_PROMPT = "模仿："
TITLE_PROMPT = "作诗："
EOS_TOKEN = '</s>'
def build_dataset_df(df, include_author=True):
  dfc = df.copy()
  if include_author:
    dfc['source_text'] = TITLE_PROMPT + df['s_title_trim'] + EOS_TOKEN + AUTHOR_PROMPT + df['s_author_trim']
  else:
    dfc['source_text'] = TITLE_PROMPT + df['s_title_trim']
  dfc['target_text'] = df['s_content_trim']
  dfc = dfc[['source_text', 'target_text']]
  return dfc

In [46]:
df_author_title_content = build_dataset_df(my_df, True)
df_author_title_content[100:105]

,source_text,target_text
100,作诗：守岁</s>模仿：高宗皇帝,今宵冬律尽，来朝丽景新。花余凝地雪，条含煖吹分。
101,作诗：九月九日幸临渭亭登高得秋</s>模仿：中宗皇帝,九月正乘秋，三杯兴已周。泛桂迎尊满，吹花向酒浮。
102,作诗：登骊山高顶寓目</s>模仿：中宗皇帝,四郊秦汉国，八水帝王都。阊阖雄里闬，城阙壮规模。
103,作诗：幸秦始皇陵</s>模仿：中宗皇帝,眷言君失德，骊邑想秦余。政烦方改篆，愚俗乃焚书。
104,作诗：立春日游苑迎春</s>模仿：中宗皇帝,神皐福地三秦邑，玉台金阙九仙家。寒光犹恋甘泉树，淑景偏临建始花。


In [47]:
df_title_content = build_dataset_df(my_df, False)
df_title_content[100:105]

,source_text,target_text
100,作诗：守岁,今宵冬律尽，来朝丽景新。花余凝地雪，条含煖吹分。
101,作诗：九月九日幸临渭亭登高得秋,九月正乘秋，三杯兴已周。泛桂迎尊满，吹花向酒浮。
102,作诗：登骊山高顶寓目,四郊秦汉国，八水帝王都。阊阖雄里闬，城阙壮规模。
103,作诗：幸秦始皇陵,眷言君失德，骊邑想秦余。政烦方改篆，愚俗乃焚书。
104,作诗：立春日游苑迎春,神皐福地三秦邑，玉台金阙九仙家。寒光犹恋甘泉树，淑景偏临建始花。


In [48]:
merged_df = pd.concat([df_author_title_content, df_title_content])

In [49]:
merged_df = merged_df.sample(frac=1.)
merged_df

,source_text,target_text
81860,作诗：送宁秀才过溪口占</s>模仿：彭汝砺,邂逅吾所乐，爰复惜分携。系帆且少留，聊慰吾所思。
184672,作诗：题汤正仲墨梅,闲庵笔底回三春，平生爱爲梅写真。只今龙钟已八十，双瞳挟电摇青旻。
203889,作诗：和周居易见寄韵</s>模仿：柴望,十年爲客上长安，人指冰山不会寒。开口尽言投老易，到头只是挂冠难。
62933,作诗：简翁都官,倦游公府曳长裾，笑上扁舟指旧庐。自有文章真杞梓，不须雕琢是璠玙。
52685,作诗：送刁安丰</s>模仿：梅尧臣,尝游芍陂上，颇见楚人爲。水有鸟鱼美，土多姜芋宜。
...,...,...
81882,作诗：衢州道中</s>模仿：彭汝砺,登山复降山，仆膝良已酸。出溪复入溪，仆衣未尝干。
75343,作诗：次韵穆父兄见寄,乌衣巷裏走双轮，正是家山二月春。明日湖平定归去，蓬莱还见谪仙人。
197575,作诗：即事十首其,窥园何物愠无端，疥手偷将玉雪团。不道主林神忌讳，鸟乌声乐并催残。
99956,作诗：大雪寄许彦周宣教法弟</s>模仿：释德洪,湘西雪连日，荒寒发明鲜。谁持华藏界，堕我宴坐边。


## Modeling

In [50]:
# Quiet install simple T5 package
!pip install -q simplet5 &> /dev/null

In [51]:
import torch
from simplet5 import SimpleT5
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [52]:
torch.cuda.empty_cache() 

In [53]:
class MengziSimpleT5(SimpleT5):
  def __init__(self) -> None:
    super().__init__()
    self.device = torch.device("cuda")

  def load_my_model(self, use_gpu: bool = True):
    self.tokenizer = T5Tokenizer.from_pretrained("Langboat/mengzi-t5-base")
    self.model = T5ForConditionalGeneration.from_pretrained("Langboat/mengzi-t5-base")

In [54]:
model = MengziSimpleT5()
model.load_my_model()
model.model = model.model.to('cuda')

Downloading:   0%|          | 0.00/708k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/659 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/945M [00:00<?, ?B/s]

In [55]:
model.tokenizer("桥形通汉上，峰势接云危。</s>烟霞交隐映，花鸟自参差。")

{'input_ids': [1012, 955, 406, 921, 23, 3, 1440, 2180, 799, 355, 4008, 4, 1, 1448, 4152, 690, 3934, 4990, 3, 17544, 178, 2572, 769, 4, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [56]:
model.tokenizer.decode([1012, 955, 406, 921, 23, 3, 1440, 2180, 799, 355, 4008, 4, 1, 1448, 4152, 690, 3934, 4990, 3, 17544, 178, 2572, 769, 4, 1])

'桥形通汉上,峰势接云危。</s> 烟霞交隐映,花鸟自参差。</s>'

In [57]:
from sklearn.model_selection import train_test_split
merged_df = merged_df.sample(frac=1) # Shuffle
train_df, eval_df = train_test_split(merged_df, test_size=0.02)

In [58]:
print("train", len(train_df), "eval", len(eval_df))

train 442685 eval 9035


In [71]:
model.train(train_df=train_df,
            eval_df=eval_df, 
            source_max_token_len=(len(TITLE_PROMPT) + MAX_TITLE_CHAR +  1 + len(AUTHOR_PROMPT) + MAX_AUTHOR_CHAR),
            target_max_token_len=MAX_CONTENT_CHAR, 
            batch_size=256,
            max_epochs=2,
            use_gpu=True,
            outputdir="/content/drive/MyDrive/ML/Models/t5-poem-v2.1")

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 247 M 
-----------------------------------------------------
247 M     Trainable params
0         Non-trainable params
247 M     Total params
990.311   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.seed:Global seed set to 42
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [72]:
def poem(title_str, opt_author=None, model=model,
         is_input_traditional_chinese=False,
         num_beams=2):
  model.model = model.model.to('cuda')
  if opt_author:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR] + EOS_TOKEN + AUTHOR_PROMPT + opt_author[:MAX_AUTHOR_CHAR]
  else:
    in_request = TITLE_PROMPT + title_str[:MAX_TITLE_CHAR]
  if is_input_traditional_chinese:
    in_request = chinese_converter.to_simplified(in_request)
  out = model.predict(in_request,
                      max_length=MAX_CONTENT_CHAR,
                      num_beams=num_beams)[0].replace(",", "，")
  if is_input_traditional_chinese:
    out = chinese_converter.to_traditional(out)
    print(f"標題： {in_request.replace('</s>', ' ')}\n詩歌： {out}")
  else:
    print(f"标题： {in_request.replace('</s>', ' ')}\n诗歌： {out}")

In [73]:
for title in ['秋思', "百花", '佳人有约']:
  # Empty author means general style
  for author in ['', "杜甫", "李白", "李清照", "苏轼"]:
    poem(title, author)
  print()

标题： 作诗：秋思
诗歌： 秋风吹我衣，夜雨湿我衣。庭树有鸣凤，露草无归。
标题： 作诗：秋思 模仿：杜甫
诗歌： 西风萧飒木叶脱，夜雨潇潇江水流。秋思悠悠日复日，客愁迢递心悠悠。
标题： 作诗：秋思 模仿：李白
诗歌： 秋色满南国，客心日忡忡。故人千里别，归思万山深。
标题： 作诗：秋思 模仿：李清照
诗歌： 西风萧瑟吹秋声，木叶惊飞雁阵横。万里客愁归不得，孤舟夜泊月明时。
标题： 作诗：秋思 模仿：苏轼
诗歌： 西风吹叶脱，落叶满庭除。老树无留影，寒虫有断肠。

标题： 作诗：百花
诗歌： 百花竞芳菲，春至辄不获。采之在庭除，芬馥何足道。
标题： 作诗：百花 模仿：杜甫
诗歌： 春色满江国，百花如洛花。春风能几日，爲我洗尘埃。
标题： 作诗：百花 模仿：李白
诗歌： 百花丛裏尽，春色满东城。不觉春风老，还随衆卉生。
标题： 作诗：百花 模仿：李清照
诗歌： 百花头上开，红白纷纷照眼明。春色不随人意浅，春风应与我心清。
标题： 作诗：百花 模仿：苏轼
诗歌： 百花丛裏开，不待春风至。春来何所如，但见日华好。

标题： 作诗：佳人有约
诗歌： 佳人有约在烟霞，笑语相呼到日斜。莫道此中无好伴，只缘春色是天涯。
标题： 作诗：佳人有约 模仿：杜甫
诗歌： 佳人美天姿，与月共婵娟。春日在何许，人间无此妍。
标题： 作诗：佳人有约 模仿：李白
诗歌： 玉人有约在蓬莱，云是瑶台第几回。今日相逢便相笑，不知春色去无来。
标题： 作诗：佳人有约 模仿：李清照
诗歌： 佳人有约在烟霄，相约寻春到绮寮。红烛夜凉窥玉佩，碧云秋淡隔纱桥。
标题： 作诗：佳人有约 模仿：苏轼
诗歌： 佳人有约在瑶台，云是当年玉女来。今日丹山应咫尺，何年携手上青霄。



In [74]:
for title in ['冬雪']:
  for author in  ['', "杜甫"]:
    for num_beams in (2, 3, 5, 10, 20, 50, 100, 200):    
      print(f"num beams: {num_beams}")
      poem(title, author, num_beams=num_beams)
    print("-"*80)

num beams: 2
标题： 作诗：冬雪
诗歌： 朔风吹雪满空山，岁晚天寒客路难。不似梅花开较早，只愁春色在阑干。
num beams: 3
标题： 作诗：冬雪
诗歌： 冬雪飞不到，北风寒更吹。朔风吹不断，万木号空垂。
num beams: 5
标题： 作诗：冬雪
诗歌： 腊后雪犹在，春前雪未消。不知梅子落，但觉麦花骄。
num beams: 10
标题： 作诗：冬雪
诗歌： 朔风吹雪满山城，万木号枯欲断魂。老去光阴能几许，春来消息亦难论。
num beams: 20
标题： 作诗：冬雪
诗歌： 寒气侵人冷似冰，晓来飞雪满空庭。不须更上高楼望，自有琼瑶万顷青。
num beams: 50
标题： 作诗：冬雪
诗歌： 去年冬雪未全消，今岁春寒犹未消。麦陇连云青似染，梅天带雪白于膏。
num beams: 100
标题： 作诗：冬雪
诗歌： 腊后寒犹在，冬来暖未回。雪因风力重，人爲岁华来。
num beams: 200
标题： 作诗：冬雪
诗歌： 岁晚风霜急，冬深雪意迟。自怜爲客久，宁与故人期。
--------------------------------------------------------------------------------
num beams: 2
标题： 作诗：冬雪 模仿：杜甫
诗歌： 冬雪不盈尺，朔风来满空。江天无片月，野渡有孤鸿。
num beams: 3
标题： 作诗：冬雪 模仿：杜甫
诗歌： 冬雪未全消，春寒犹未已。江云不辨朝，山色欲明日。
num beams: 5
标题： 作诗：冬雪 模仿：杜甫
诗歌： 朔风吹雪满平川，漠漠飞云万里天。白草江边人迹绝，乱山深处马行偏。
num beams: 10
标题： 作诗：冬雪 模仿：杜甫
诗歌： 冬雪未全销，春寒犹未回。北风来不断，南雪去还来。
num beams: 20
标题： 作诗：冬雪 模仿：杜甫
诗歌： 朔雪连三白，冬云覆四溟。北风惊岁晚，南雪伴春深。
num beams: 50
标题： 作诗：冬雪 模仿：杜甫
诗歌： 腊雪连三白，冬云暗九垓。朔风从北至，积雪向南来。
num beams: 100
标题： 作诗：冬雪 模仿：杜甫
诗歌： 瘴地冬无雪，江城岁有冰。朔风黄叶密，寒气白沙凝。
num beams: 200
标题： 作诗：冬雪 模仿：杜甫
诗歌： 朔风吹雪满江滨，塞北